# Life Expectancy

This notebook loads and visualizes life expectancy data by location (1960–2022)

- Denise Case
- 2025-05

In [133]:
# Imports (Once at the top of the file)
from pathlib import Path
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

# Define Constant Paths
NOTEBOOK_DIR = Path().resolve()
ROOT_DIR = NOTEBOOK_DIR.parent
DATA_DIR = ROOT_DIR / "data"
DATA_FILE = DATA_DIR / "raw" / "API_SP.DYN.LE00.IN_DS2_en_csv_v2_369933.csv"


## Load Data and Inspect

In [134]:
df_raw = pd.read_csv(DATA_FILE, skiprows=4)
df_raw.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,64.049000,64.215000,64.602000,64.944000,65.303000,65.615000,...,75.540000,75.620000,75.880000,76.019000,75.406000,73.655000,76.226000,76.353000,NaN,NaN
1,Africa Eastern and Southern,AFE,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,44.169257,44.468838,44.877890,45.160583,45.535695,45.770723,...,62.167981,62.591275,63.330691,63.857261,63.766484,62.979999,64.487020,65.146291,NaN,NaN
2,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.799000,33.291000,33.757000,34.201000,34.673000,35.124000,...,62.646000,62.406000,62.443000,62.941000,61.454000,60.417000,65.617000,66.035000,NaN,NaN
3,Africa Western and Central,AFW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.779636,38.058956,38.681792,38.936918,39.194580,39.479784,...,56.392452,56.626439,57.036976,57.149847,57.364425,57.362572,57.987813,58.855722,NaN,NaN
4,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.933000,36.902000,37.168000,37.419000,37.704000,37.968000,...,61.619000,62.122000,62.622000,63.051000,63.116000,62.958000,64.246000,64.617000,NaN,NaN


## Clean and Transform the Data

In [135]:
# Drop rows missing country name
df = df_raw.dropna(subset=["Country Name"]).copy()

# Keep only year columns
year_cols = [col for col in df.columns if col.isdigit()]
df = df[["Country Name"] + year_cols]

# Set country as columns, years as rows
df.set_index("Country Name", inplace=True)
df.head()


,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,64.049000,64.215000,64.602000,64.944000,65.303000,65.615000,66.126000,66.385000,66.744000,67.098000,...,75.405000,75.540000,75.620000,75.880000,76.019000,75.406000,73.655000,76.226000,76.353000,NaN
Africa Eastern and Southern,44.169257,44.468838,44.877890,45.160583,45.535695,45.770723,45.765725,46.440745,46.738632,46.977476,...,61.713031,62.167981,62.591275,63.330691,63.857261,63.766484,62.979999,64.487020,65.146291,NaN
Afghanistan,32.799000,33.291000,33.757000,34.201000,34.673000,35.124000,35.583000,36.042000,36.510000,36.979000,...,62.270000,62.646000,62.406000,62.443000,62.941000,61.454000,60.417000,65.617000,66.035000,NaN
Africa Western and Central,37.779636,38.058956,38.681792,38.936918,39.194580,39.479784,39.719248,39.529359,40.258890,40.567814,...,56.038336,56.392452,56.626439,57.036976,57.149847,57.364425,57.362572,57.987813,58.855722,NaN
Angola,37.933000,36.902000,37.168000,37.419000,37.704000,37.968000,38.258000,38.616000,38.968000,39.329000,...,61.042000,61.619000,62.122000,62.622000,63.051000,63.116000,62.958000,64.246000,64.617000,NaN


## Transpose for Time-Based Charts

We want:
- Rows: Years (1960–2022)
- Columns: Location/Country Names
- Values: Life expectancy at birth

In [136]:
# Transpose the DataFrame to have years as rows and locations (countries) as columns
df = df.T

# Set the index name to "Year" and remove the column name
df.index.name = "Year"  

# Remove the name of the columns ("Country Names") to simplify
df.columns.name = None
df = df.astype(float)
df.head()

,Aruba,Africa Eastern and Southern,Afghanistan,Africa Western and Central,Angola,Albania,Andorra,Arab World,United Arab Emirates,Argentina,...,Virgin Islands (U.S.),Viet Nam,Vanuatu,World,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
Year,,,,,,,,,,,,,,,,,,,,,
1960,64.049,44.169257,32.799,37.779636,37.933,56.413,72.094,44.794056,50.651,64.242,...,62.743537,58.046,49.841,50.941976,55.153,48.702,33.462,52.575,50.648,53.492
1961,64.215,44.468838,33.291,38.058956,36.902,57.488,72.573,45.545371,51.596,64.631,...,63.121707,58.461,50.384,52.797237,55.738,49.883,34.058,53.067,51.041,53.966
1962,64.602,44.877890,33.757,38.681792,37.168,58.494,72.993,45.993769,52.546,64.618,...,62.668024,58.204,50.931,55.286077,56.316,48.378,33.669,53.566,51.331,54.453
1963,64.944,45.160583,34.201,38.936918,37.419,59.479,73.298,46.979338,53.509,64.855,...,61.434707,56.897,51.485,55.652229,56.861,50.098,33.431,53.895,51.605,54.942
1964,65.303,45.535695,34.673,39.194580,37.704,60.404,73.624,47.570034,54.495,64.816,...,64.124927,57.473,51.969,56.096698,54.043,51.119,34.907,54.215,51.170,55.431


## Compare Life Expectancy at Birth between Two Locations

Use the dropdown boxes to select your choices.

In [137]:
# Dropdown widgets for location selection
location_list = sorted(df.columns.tolist())
location1 = widgets.Dropdown(options=location_list, description="Location 1:", value=location_list[0])
location2 = widgets.Dropdown(options=location_list, description="Location 2:", value=location_list[13])

# Define custom styles and markers
style_map = {
    "solid": "solid",
    "dash": "dash",
    "dot": "dot"
}
marker_map = {
    "circle": "circle",
    "square": "square",
    "diamond": "diamond"
}
color_map = {
    location1.value: "royalblue",
    location2.value: "green"
}



def plot_comparison(c1, c2):
    fig = go.Figure()

    for i, country in enumerate([c1, c2]):
        fig.add_trace(
            go.Scatter(
                x=df.index.astype(int),
                y=df[country],
                mode="lines+markers",
                name=country,
                line=dict(
                    dash=list(style_map.values())[i],
                    width=2,
                    color=color_map.get(country, None)
                ),
                marker=dict(
                    symbol=list(marker_map.values())[i],
                    size=6
                )
            )
        )

    fig.update_layout(
        title=f"Life Expectancy Comparison:\n{c1} vs {c2}",
        xaxis_title="Year",
        yaxis_title="Life Expectancy at Birth (Years)",
        height=500
    )

    fig.show()

widgets.interact(plot_comparison, c1=location1, c2=location2);


interactive(children=(Dropdown(description='Location 1:', options=('Afghanistan', 'Africa Eastern and Southern…